# Cleaning Master

### Cleaning "Border_Crossing_Entry_Data.zip"

In [1]:
# load helpful packages 
import pandas as pd # data processing
import numpy as np # linear algebra

Let's first import the data and let's see what the dataset is like

In [2]:
# load dataset
df=pd.read_csv("../data/Border_Crossing_Entry_Data.zip")
# df = pd.read_csv("../data/data_with_position.zip") to be delated
# "each row has a counting ("Value") for the crossing method ("Measure")"
df.head()

,Port Name,State,Port Code,Border,Date,Measure,Value
0,Warroad,Minnesota,3423,US-Canada Border,06/01/2019 12:00:00 AM,Truck Containers Full,133
1,Warroad,Minnesota,3423,US-Canada Border,06/01/2019 12:00:00 AM,Truck Containers Empty,298
2,Warroad,Minnesota,3423,US-Canada Border,06/01/2019 12:00:00 AM,Personal Vehicles,10383
3,Warroad,Minnesota,3423,US-Canada Border,06/01/2019 12:00:00 AM,Personal Vehicle Passengers,19459
4,Warroad,Minnesota,3423,US-Canada Border,06/01/2019 12:00:00 AM,Pedestrians,2


Now we'll take look at the data structure

In [3]:
# check data types of features
df.dtypes

Port Name    object
State        object
Port Code     int64
Border       object
Date         object
Measure      object
Value         int64
dtype: object

We see that the data types need some reformating

In [4]:
# convert 'Date' to date format
df['Date'] = pd.to_datetime(df['Date'])
# recheck data type of 'Date'
df.dtypes

Port Name            object
State                object
Port Code             int64
Border               object
Date         datetime64[ns]
Measure              object
Value                 int64
dtype: object

In [5]:
df.head()

,Port Name,State,Port Code,Border,Date,Measure,Value
0,Warroad,Minnesota,3423,US-Canada Border,2019-06-01,Truck Containers Full,133
1,Warroad,Minnesota,3423,US-Canada Border,2019-06-01,Truck Containers Empty,298
2,Warroad,Minnesota,3423,US-Canada Border,2019-06-01,Personal Vehicles,10383
3,Warroad,Minnesota,3423,US-Canada Border,2019-06-01,Personal Vehicle Passengers,19459
4,Warroad,Minnesota,3423,US-Canada Border,2019-06-01,Pedestrians,2


In [6]:
# Look for missing values
df.isnull().any()

Port Name    False
State        False
Port Code    False
Border       False
Date         False
Measure      False
Value        False
dtype: bool

We don't have any missing values. Now let's check the number of different values in each columns

In [7]:
print('Attribute '+ 'Values')
for i in df.columns:
    print( i,len(df.loc[:,i].unique()) )

Attribute Values
Port Name 116
State 15
Port Code 117
Border 2
Date 282
Measure 12
Value 53725


In [8]:
# check if number of unique elements in "Port Code" and "Port Name" equal 
print('port names : {}' .format(len(df['Port Name'].unique())))
print('port codes : {}' .format(len(df['Port Code'].unique())))

port names : 116
port codes : 117


We see that there are one more port code than port name so we have to fix that.

In [9]:
ports = df[['Port Code','Port Name']].drop_duplicates()
ports[ports['Port Name'].duplicated(keep=False)]

,Port Code,Port Name
525,3302,Eastport
562,103,Eastport


In [10]:
df.loc[df['Port Name'] == 'Eastport'].drop_duplicates(['Port Code'], keep='first')

,Port Name,State,Port Code,Border,Date,Measure,Value
525,Eastport,Idaho,3302,US-Canada Border,2019-06-01,Bus Passengers,43
562,Eastport,Maine,103,US-Canada Border,2019-06-01,Bus Passengers,179


It is because there are 2 ports with the same name in different state so we will fix that by changing the port names

In [11]:
# change 'Port Name' of Eastport Idaho and Eastport Maine
df.loc[(df['Port Name'] == 'Eastport') & (df['State'] == 'Idaho'), 'Port Name'] = 'Eastport_ID'
df.loc[(df['Port Name'] == 'Eastport') & (df['State'] == 'Maine'), 'Port Name'] = 'Eastport_MA'
df.loc[(df['Port Code'] == 3302) | (df['Port Code'] == 103)].drop_duplicates(['Port Code'], keep='first')

,Port Name,State,Port Code,Border,Date,Measure,Value
525,Eastport_ID,Idaho,3302,US-Canada Border,2019-06-01,Bus Passengers,43
562,Eastport_MA,Maine,103,US-Canada Border,2019-06-01,Bus Passengers,179


In [12]:
# recheck if number of unique elements in "Port Code" and "Port Name" equal 
print('port names : {}' .format(len(df['Port Name'].unique())))
print('port codes : {}' .format(len(df['Port Code'].unique())))

port names : 117
port codes : 117


The problem is now solved.

We will now add 2 columns for the Year and Month extracted from the date column. It will be used later on in the EDA.

In [13]:
df['Date'] = pd.to_datetime(df['Date'])
df['Year'] = df['Date'].apply(lambda x : x.year)
month_mapper = {1:'Jan', 2:'Feb', 3:'Mar', 4:'Apr', 5:'May', 6:'Jun'
               ,7:'Jul', 8:'Aug', 9:'Sep' ,10:'Oct', 11:'Nov', 12:'Dec'}
df['Month'] = df['Date'].apply(lambda x : x.month).map(month_mapper)
del month_mapper

In [14]:
df.head()

,Port Name,State,Port Code,Border,Date,Measure,Value,Year,Month
0,Warroad,Minnesota,3423,US-Canada Border,2019-06-01,Truck Containers Full,133,2019,Jun
1,Warroad,Minnesota,3423,US-Canada Border,2019-06-01,Truck Containers Empty,298,2019,Jun
2,Warroad,Minnesota,3423,US-Canada Border,2019-06-01,Personal Vehicles,10383,2019,Jun
3,Warroad,Minnesota,3423,US-Canada Border,2019-06-01,Personal Vehicle Passengers,19459,2019,Jun
4,Warroad,Minnesota,3423,US-Canada Border,2019-06-01,Pedestrians,2,2019,Jun


## Adding Positions to "Border_Crossing_Entry_Data.zip"

We will now add positions of the different crossing points to the dataset.

In [15]:
from geopy.geocoders import Nominatim

In [16]:
df.head()

,Port Name,State,Port Code,Border,Date,Measure,Value,Year,Month
0,Warroad,Minnesota,3423,US-Canada Border,2019-06-01,Truck Containers Full,133,2019,Jun
1,Warroad,Minnesota,3423,US-Canada Border,2019-06-01,Truck Containers Empty,298,2019,Jun
2,Warroad,Minnesota,3423,US-Canada Border,2019-06-01,Personal Vehicles,10383,2019,Jun
3,Warroad,Minnesota,3423,US-Canada Border,2019-06-01,Personal Vehicle Passengers,19459,2019,Jun
4,Warroad,Minnesota,3423,US-Canada Border,2019-06-01,Pedestrians,2,2019,Jun


We'll add a new column with coordinates of the ports. These coordinates will allow to plot the ports on a map.

We create a new locator with geopy:

In [17]:
locator = Nominatim(user_agent="myGeocoder")

In [18]:
nom = Nominatim(user_agent="myGeocoder",timeout=2)

We create an array with all the names of the ports
and add ",USA" at the end of every port name to make sure to find we the right one in the USA.

In [19]:
df['Port_state'] = df['Port Name'] + "," + df['State']
lieux = df['Port_state'].map(lambda x : x).unique()
lieux = lieux + ",USA"
#lieux

We find the geocode of every port and store them in a dictionnary.

In [20]:
coordinates = {}

def do_geocode(address):
    try:
        return nom.geocode(address)
    except:
        return do_geocode(address)
    
for lieu in lieux:
    coordinates[lieu] = do_geocode(lieu)
    #print(do_geocode(lieu))

We compute a new dictionnary with the exact name of the ports (whitout USA at the end) and the coordinates.

In [21]:
portAndCoordinates = {}
for key in coordinates:
    k = key.split(',', 1)[0]
    if coordinates[key] is None:
        portAndCoordinates[k] = None
    else:
        portAndCoordinates[k] = (coordinates[key].latitude,coordinates[key].longitude)

We now look for missing values in the ports coordinates

In [22]:
missings = []
for key in portAndCoordinates:
    if portAndCoordinates[key] is None:
        missings.append(key)
print(missings)

['International Falls-Ranier', 'Dalton Cache', 'Eastport_ID', 'Eastport_MA']


We see that these 7 ports have no coordinates. We'll add them manually.

In [39]:
portAndCoordinates['International Falls-Ranier'] = (48.614892, -93.352057)
portAndCoordinates['Dalton Cache'] = (59.470921, -136.419291)
portAndCoordinates['Eastport_MA'] = (44.911752, -66.985965)
portAndCoordinates['Eastport_ID'] = (48.999904,-116.180196)

We add the coordinates to the data as a new column

In [40]:
df["position"] = df['Port Name'].apply(lambda loc:portAndCoordinates[loc])

In [41]:
df.head() 

,Port Name,State,Port Code,Border,Date,Measure,Value,Year,Month,position
0,Warroad,Minnesota,3423,US-Canada Border,2019-06-01,Truck Containers Full,133,2019,6,"(48.905266, -95.314404)"
1,Warroad,Minnesota,3423,US-Canada Border,2019-06-01,Truck Containers Empty,298,2019,6,"(48.905266, -95.314404)"
2,Warroad,Minnesota,3423,US-Canada Border,2019-06-01,Personal Vehicles,10383,2019,6,"(48.905266, -95.314404)"
3,Warroad,Minnesota,3423,US-Canada Border,2019-06-01,Personal Vehicle Passengers,19459,2019,6,"(48.905266, -95.314404)"
4,Warroad,Minnesota,3423,US-Canada Border,2019-06-01,Pedestrians,2,2019,6,"(48.905266, -95.314404)"


In [42]:
df = df.drop(columns='Port_state')

KeyError: "['Port_state'] not found in axis"

In [43]:
df['Month'] = df['Date'].dt.month
df.head()

,Port Name,State,Port Code,Border,Date,Measure,Value,Year,Month,position
0,Warroad,Minnesota,3423,US-Canada Border,2019-06-01,Truck Containers Full,133,2019,6,"(48.905266, -95.314404)"
1,Warroad,Minnesota,3423,US-Canada Border,2019-06-01,Truck Containers Empty,298,2019,6,"(48.905266, -95.314404)"
2,Warroad,Minnesota,3423,US-Canada Border,2019-06-01,Personal Vehicles,10383,2019,6,"(48.905266, -95.314404)"
3,Warroad,Minnesota,3423,US-Canada Border,2019-06-01,Personal Vehicle Passengers,19459,2019,6,"(48.905266, -95.314404)"
4,Warroad,Minnesota,3423,US-Canada Border,2019-06-01,Pedestrians,2,2019,6,"(48.905266, -95.314404)"


We check if data are still missing

In [44]:
df.isnull().sum()

Port Name    0
State        0
Port Code    0
Border       0
Date         0
Measure      0
Value        0
Year         0
Month        0
position     0
dtype: int64

Now the we cleaned the dataset and added the positions we will export it:

In [45]:
export_csv = df.to_csv (r'../data/DataWithLocationCleaned.csv.gz', compression='gzip',index = None, header=True)